In [2]:
# Importing necessary packages, may need to download cyvcf2 and numpy joblib to enhance performance
%pylab inline
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from cyvcf2 import VCF

# Load phenotype data
ptdata = pd.read_csv(os.path.expanduser("~/public/lab3/lab3_gwas.phen"), delim_whitespace=True, names=["FID","IID","LDL"])

# Load genotype data
vcf_file = os.path.expanduser("~/public/lab3/lab3_gwas.vcf.gz")
vcf_reader = VCF(vcf_file)

# Extract sample IDs
samples = vcf_reader.samples

# Extract sample IDs & associated genotype data
genotype_data = []
variant_ids = []

for record in vcf_reader:
    variant_ids.append(record.ID)
    genotypes = [gt[0] + gt[1] for gt in record.genotypes]
    genotype_data.append(genotypes)

# Convert genotype data to pandas DataFrame
genotype_data = np.array(genotype_data).T
genotype_df = pd.DataFrame(genotype_data, columns=variant_ids, index=samples)

Populating the interactive namespace from numpy and matplotlib


/opt/conda/lib/python3.9/site-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['record']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [4]:
# Filter SNPs based on criteria used by the Wellcome Trust Case Control Consortium (WTCCC) when conducting GWAS
# The criteria for retaining a SNP are: HWE P-value ≥ 5.7 × 10−7, MSP ≤5% if MAF ≥ 5%, MSP ≤ 1%
def filter_snps(genotype_df, maf_threshold = 0.05, missingness_proportion = 0.05):
    
    # Calculate minor alllele frequency (MAF)
    allele_freq = genotype_df.apply(lambda x: np.sum(x) / (2 * len(x)), axis = 0)
    maf = np.minimum(allele_freq, 1 - allele_freq)
    
    # Filter SNPs based on MAF
    genotype_df = genotype_df.loc[:, maf > maf_threshold]
    
    # Filter SNPs by missingness proportion (MSP)
    missingness = genotype_df.isnull().mean()
    genotype_df = genotype_df.loc[:, missingness < missingness_proportion]
    
    return genotype_df

# Follow same guidelines usde by WTCCC to filter individuals based on missingness
def filter_indivs(genotype_df, missingness_proportion = 0.05):
    missingness = genotype_df.isnull().mean(axis = 1)
    genotype_df = genotype_df.loc[missingness < missingness_proportion, :]
    return genotype_df

# Perform QC uby filtering SNPs, individuals, and based on HWE p-value
genotype_df = filter_snps(genotype_df)
genotype_df = filter_indivs(genotype_df)

# Display the first few rows of the filtered genotype data
genotype_df.head()


,rs11252127,rs7909677,rs11591988,rs12768206,rs10904561,rs7917054,rs7906287,rs9419557,rs9286070,rs11253562,...,rs2606358,rs2739260,rs2229949,rs3750508,rs3750510,rs9777369,rs11137376,rs17583562,rs11137379,rs9314655
NA06984,0,0,0,1,0,1,1,0,0,1,...,2,2,2,0,2,0,0,0,0,0
NA06989,1,1,0,1,1,1,2,0,0,0,...,1,1,2,1,2,0,0,1,1,0
NA12878,0,0,0,1,0,1,1,0,0,1,...,1,1,2,1,2,0,0,0,0,0
NA18489,0,0,0,1,0,1,1,0,1,0,...,0,0,1,0,2,1,1,0,0,1
NA18504,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,2,0,0,0,2


In [6]:
from joblib import Parallel, delayed
from scipy.stats import chi2_contingency
# Function to calculate HWE p-value
def hwe_p_value(genotype_counts):
    obs_homo1 = genotype_counts.get(0, 0)
    obs_het = genotype_counts.get(1, 0)
    obs_homo2 = genotype_counts.get(2, 0)
    n = obs_homo1 + obs_het + obs_homo2

    if n == 0:
        return 1.0  # Avoid division by zero

    p = (2 * obs_homo1 + obs_het) / (2 * n)
    q = 1 - p

    exp_homo1 = n * p * p
    exp_het = 2 * n * p * q
    exp_homo2 = n * q * q

    # Skip SNPs where the expected count for any genotype is less than 1
    if min(exp_homo1, exp_het, exp_homo2) < 1:
        return 1.0

    observed = [obs_homo1, obs_het, obs_homo2]
    expected = [exp_homo1, exp_het, exp_homo2]

    chi2, p_value = chi2_contingency([observed, expected])[:2]

    return p_value

# Function to filter SNPs based on HWE
def filter_snps_hwe(genotype_df, hwe_threshold=1e-6, n_jobs=-1):
    def process_snp(snp):
        genotypes = genotype_df[snp].dropna()
        genotype_counts = genotypes.value_counts().to_dict()
        p_value = hwe_p_value(genotype_counts)
        return (snp, p_value)

    results = Parallel(n_jobs=n_jobs)(
        delayed(process_snp)(snp) for snp in genotype_df.columns
    )

    snps_to_keep = [snp for snp, p_value in results if p_value >= hwe_threshold]

    return genotype_df[snps_to_keep]

# Apply HWE filtering
filtered_genotype_df = filter_snps_hwe(genotype_df, hwe_threshold=1e-6)

# Display the shape of the filtered genotype DataFrameg
print(f"Shape of genotype data before HWE filtering: {genotype_df.shape}")
print(f"Shape of genotype data after HWE filtering: {filtered_genotype_df.shape}")

exception calling callback for <Future at 0x7f0694f91340 state=finished raised BrokenProcessPool>
joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py", line 404, in _process_worker
    call_item = call_queue.get(block=True, timeout=timeout)
  File "/opt/conda/lib/python3.9/multiprocessing/queues.py", line 108, in get
    if not self._rlock.acquire(block, timeout):
KeyboardInterrupt
"""

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/joblib/externals/loky/_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "/opt/conda/lib/python3.9/site-packages/joblib/parallel.py", line 359, in __call__
    self.parallel.dispatch_next()
  File "/opt/conda/lib/python3.9/site-packages/joblib/parallel.py", line 792, in dispatch_next
    if not self.dispat

KeyboardInterrupt: 

In [ ]:
# THE ONE THAT WORKED
from scipy.stats import t
from sklearn.linear_model import LinearRegression
# Function for association testing using linear regression
def linear_regression_association_test(genotype_df, phenotype_data):
    results = []

    # Iterate over each SNP in the filtered genotype DataFrame
    for snp in genotype_df.columns:
        # Extract genotype values for the current SNP
        genotype_values = genotype_df[snp].values.reshape(-1, 1)

        # Extract phenotype values corresponding to the genotypes
        phenotype_values = phenotype_data["LDL"].values.reshape(-1, 1)

        # Fit linear regression model
        model = LinearRegression().fit(genotype_values, phenotype_values)

        # Get the coefficient and standard error
        coef = model.coef_[0][0]
        std_err = np.sqrt(np.mean((phenotype_values - model.predict(genotype_values))**2))  # Standard error of the coefficient

        # Calculate the t-statistic and its p-value
        t_statistic = coef / std_err
        dof = len(phenotype_values) - 2  # Degrees of freedom
        p_value = 2 * (1 - t.cdf(abs(t_statistic), df=dof))

        # Append SNP and p-value to results
        results.append((snp, p_value))

    # Create DataFrame from results
    results_df = pd.DataFrame(results, columns=['SNP', 'p-value'])
    
    return results_df

results = linear_regression_association_test(genotype_data, phenotype_data)